## 케라스 모델과 레이어를 맞춤형으로 사용하기
##### url: https://www.tensorflow.org/guide/keras/custom_layers_and_models

- 목차
    - 설정
    - 레이어 클래스
        - 레이어에서 가중치와 연산 캡슐화하기
        - 모범 예제: 입력 차원이 알려지기 전까지 가중치 생성 지연
        - 재귀적으로 레이어 구성하기
        - 데이터가 모델을 통과하는 동안 생성되는 손실 값을 재귀적으로 수집하는 레이어
        - 선택적으로 레이어 직렬화 활성화하기
        - Call 메소드가 가지는 훈련 인자에 대한 특권
    - 모델 작성하기
        - 모델 클래스
        - 하나로 합치기: 엔드 투 엔드 예제
        - 객체 지향 개발 기법을 넘어서: 함수형 API
        
### 설정

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.keras.backend.clear_session() # 노트북의 상태를 쉽게 초기화할 수 있습니다.

## 레이어 클래스
### 레이어에서 가중치와 연산 캡슐화하기
사용하게 될 주요 데이터구조는 `레이어` 입니다. 레이어는 상태(레이어의 가중치,'weight')와 입력을 출력으로 변환하는 것(호출,'call', 레이어의 순전파)을 캡슐화합니다.<br><br>
여기 밀집되어 연결된 레이어가 있습니다. 이것들은 변수 `w`와 `b`로 나타내어지는 상태를 가지고 있습니다. 

In [2]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                  dtype='float32'),
                             trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.12158676  0.04798947 -0.03186572  0.01187759]
 [ 0.12158676  0.04798947 -0.03186572  0.01187759]], shape=(2, 4), dtype=float32)


`w`와 `b`가 자동적으로 레이어 속성 집합으로써 추적된다는 것을 확인해보세요.

In [3]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

또한, `add_weight` 메소드를 사용하여 레이어가 가진 가중치에 더 빠르게 접근할 수 있습니다.

In [4]:
class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(units,),
                                initializer='zeros',
                                trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
s = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.07057144  0.05381956  0.03808735 -0.0719431 ]
 [ 0.07057144  0.05381956  0.03808735 -0.0719431 ]], shape=(2, 4), dtype=float32)
